In [8]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [9]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [10]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect,func

In [11]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [12]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)


In [13]:
# View all of the classes that automap found
Base.classes.keys()

[]

In [14]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

AttributeError: measurement

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
Measurement_first_row = session.query(Measurement).first()
Measurement_first_row.__dict__

Station_first_row = session.query(Station).first()
Station_first_row.__dict__

recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(recent_date)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
one_year_ago = dt.date(2017, 8, 23) - dt.timedelta(days=365)
#print(one_year_ago)


# Perform a query to retrieve the data and precipitation scores
date_prcp = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= one_year_ago).all()
  

# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(date_prcp, columns=['date', 'precipitation' ])
df = df.dropna(how='any')

# Sort the dataframe by date
df = df.sort_values("date")
df.head()


In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
df.plot(x='date', y='precipitation', rot=90, figsize =(15,4))
plt.xlabel("Date")
plt.ylabel("Inches")


plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
stations_count= session.query(Station).distinct().count()
print(stations_count)

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.

session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).all()

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
lowest_temp = session.query(func.min(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').all()
print(lowest_temp)

highest_temp = session.query(Measurement.station, func.max(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').all()
print(highest_temp)

avg_temp = session.query(Measurement.station, func.avg(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').all()
print(avg_temp)


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
one_year_temp = session.query(Measurement.tobs).\
    filter(Measurement.date >= one_year_ago, Measurement.station =='USC00519281').all()
one_year_temp

In [ ]:
temp_df = pd.DataFrame(one_year_temp, columns=['tobs'])
temp_df = temp_df.dropna(how='any')

# Sort the dataframe by tobs
temp_df = temp_df.sort_values("tobs")
temp_df.head()

In [ ]:
plt.hist(temp_df['tobs'], bins=12)
plt.xlabel('Temperature')
plt.ylabel('Frequency')
plt.show()

# Close session

In [ ]:
# Close Session
session.close()

NameError: name 'session' is not defined